In [3]:
import torch
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer
from transformers import BertPreTrainedModel, BertModel

import pandas as pd
import numpy as np
import os

from sklearn.metrics import classification_report

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  DEVICE = 'cpu'
else:
  DEVICE = 'cuda:0'

In [4]:
df = pd.read_csv('./data/olid-training-v1.0.tsv',delimiter="\t")

print(f'Number of training samples: {len(df)}')

df.head()

Number of training samples: 13240


,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [5]:
def reader_train(file_name):
    texts = []
    labels = []
    fin = open(file_name)
    title = fin.readline()
    set_a = ['NOT' , 'OFF']
    set_b = ['NULL', 'TIN', 'UNT']
    set_c = ['NULL', 'IND', 'GRP', 'OTH']
    while True:
        line = fin.readline()
        if not line:
            break
        items = line.split('\t')
        text = items[1]
        label_a = set_a.index(items[2].strip())
        label_b = set_b.index(items[3].strip())
        label_c = set_c.index(items[4].strip())

        if len(text) > 0:
            texts.append(text)
            labels.append([label_a, label_b, label_c])
            
    return {'texts':texts, 'labels':labels}

def reader_test(test_textlist, test_labellist):
    texts = []
    labels = []
    text_dict = {}
    
    # build text_dict
    for file_text in test_textlist:
        fin = open(file_text)
        title = fin.readline()
        while True:
            line = fin.readline()
            if not line:
                break
            items = line.split('\t')
            if items[0] not in text_dict:
                text_dict[items[0]] = items[1]
        fin.close()
    label_dict_list = []
    
    # build label_dict
    for i, file_label in enumerate(test_labellist):
        label_dict_list.append({})
        fin = open(file_label)
        title = fin.readline()
        while True:
            line = fin.readline()
            if not line:
                break
            items = line.split(',')
            label_dict_list[i][items[0]] = items[1]
        fin.close()    
    
    set_a = ['NOT' , 'OFF']
    set_b = ['NULL', 'TIN', 'UNT']
    set_c = ['NULL', 'IND', 'GRP', 'OTH']
    
    for idx, text in text_dict.items():
        if len(text) > 0:
            texts.append(text)
            if idx in label_dict_list[0]:
                label_a = label_dict_list[0][idx]
            else:
                label_a = 'OFF'
            if idx in label_dict_list[1]:
                label_b = label_dict_list[1][idx]
            else:
                label_b = 'NULL'
            if idx in label_dict_list[2]:
                label_c = label_dict_list[2][idx]
            else:
                label_c = 'NULL'
            
            label_a = set_a.index(label_a.strip())
            label_b = set_b.index(label_b.strip())
            label_c = set_c.index(label_c.strip())
        
            labels.append([label_a, label_b, label_c])
            
    return {'texts':texts, 'labels':labels}            

In [6]:
class OlidDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, input_set):

        self.tokenizer = tokenizer
        self.texts = input_set['texts']
        self.labels = input_set['labels']
        
    def collate_fn(self, batch):

        texts = []
        labels_a = []
        labels_b = []
        labels_c = []

        for b in batch:
            texts.append(b['text'])
            labels_a.append(b['label_a'])
            labels_b.append(b['label_b'])
            labels_c.append(b['label_c'])

        #The maximum sequence size for BERT is 512 but here the tokenizer truncate sentences longer than 128 tokens.  
        # We also pad shorter sentences to a length of 128 tokens
        encodings = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=128)
        labels = {}
        encodings['label_a'] =  torch.tensor(labels_a)
        encodings['label_b'] =  torch.tensor(labels_b)
        encodings['label_c'] =  torch.tensor(labels_c)
        
        return encodings
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
       
        item = {'text': self.texts[idx],
                'label_a': self.labels[idx][0],
                'label_b': self.labels[idx][1],
                'label_c': self.labels[idx][2]}
        return item

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# we can check the parameters of this tokenizer
tokenizer

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
trainset = reader_train('./data/olid-training-v1.0.tsv')
testset = reader_test(['./data/testset-levela.tsv','./data/testset-levelb.tsv','./data/testset-levelc.tsv'], 
                      ['./data/labels-levela.csv','./data/labels-levelb.csv','./data/labels-levelc.csv'])

train_dataset = OlidDataset(tokenizer, trainset)
test_dataset = OlidDataset(tokenizer, testset)

#returns first item as dictionnary
#print(train_dataset[0])

# put all train set into one batch for the collate_fn function
# batch = [sample for sample in train_dataset]

# encodings = train_dataset.collate_fn(batch[:10])

# for key, value in encodings.items():
#   print(f"{key}: {value.numpy().tolist()}")


In [9]:
model = BertModel.from_pretrained("bert-base-cased")

#180 M
print(f"Model size: {model.num_parameters()}")

#model summary
model

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Model size: 108310272


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [10]:
class BERT_hate_speech(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # BERT Model
        self.bert = BertModel(config)
        
        # Task A
        self.projection_a = torch.nn.Sequential(torch.nn.Dropout(0.2),
                                                torch.nn.Linear(config.hidden_size, 2))
      
        # the sigmoid activation here is not strictly necessary.
        # However, its presence here doesn't make a big difference to the model's performance
        # Why do you think that is?
        self.activation = nn.Sigmoid()


        # Task B
        # TBA
        
        # Task C
        # TBA
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # Logits A
        logits_a = self.projection_a(outputs[1])
        out =  self.activation(logits_a)
        
        return out

In [11]:
class Trainer_hate_speech(Trainer):
    def compute_loss(self, model, inputs):
        labels = {}
        labels['label_a'] = inputs.pop('label_a')
        labels['label_b'] = inputs.pop('label_b')
        labels['label_c'] = inputs.pop('label_c')

        outputs = model(**inputs)

        # TASK A
        loss_task_a = nn.CrossEntropyLoss()
        labels_a = labels['label_a']
        loss_a = loss_task_a(outputs.view(-1, 2), labels_a.view(-1))

        loss = loss_a
        
        return loss

In [12]:
def main_hate_speech():

    #call our custom BERT model and pass as parameter the name of an available pretrained model
    model = BERT_hate_speech.from_pretrained("bert-base-cased")
    
    training_args = TrainingArguments(
        output_dir='./experiment/hate_speech',
        learning_rate = 0.0001,
        logging_steps= 100,
        per_device_train_batch_size=32,
        num_train_epochs = 3,
        remove_unused_columns=False # This argument prevents the collator to drop data from our batch when customizing the data collator
    )
    trainer = Trainer_hate_speech(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/ht_bert_finetuned/')

In [ ]:
main_hate_speech()

Some weights of BERT_hate_speech were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['projection_a.1.bias', 'projection_a.1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/1242 [00:00<?, ?it/s]

{'loss': 0.5826, 'learning_rate': 9.194847020933978e-05, 'epoch': 0.24}
{'loss': 0.5557, 'learning_rate': 8.389694041867955e-05, 'epoch': 0.48}
